In [1]:
!unzip "dataset.zip" "dataset"

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
import torch

In [15]:
print(torch.__version__)

2.2.1+cu121


In [ ]:
# !pip install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch/

In [ ]:
# !pip install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/
!pip install transformers
!pip install deepspeed
!pip install wandb


In [31]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./output",  # Replace with your desired output directory
    per_device_train_batch_size=8,  # Adjust batch size as needed
    num_train_epochs=3,  # Set the number of training epochs
    fp16=True  # Enable mixed precision training with fp16
)


In [ ]:
!pip install bitsandbytes

In [ ]:
!git clone https://github.com/haotian-liu/LLaVA.git

In [ ]:
!pip install git+https://github.com/huggingface/accelerate

In [ ]:
import sys
sys.path.append('LLaVA')

from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model

model_path = "liuhaotian/llava-v1.5-7b"

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_path),
    offload_folder="llava_model"
    )


In [25]:
# Assign paths to variables
DEEPSPEED_SCRIPT = "deepspeed LLaVA/llava/train/train_mem.py"

DEEPSPEED_JSON = "LLaVA/scripts/zero2.json"  #"./scripts/zero3.json"
MODEL_NAME = "liuhaotian/llava-v1.5-7b"

DATA_PATH = "train/dataset.json" # Replace with your JSON data path
IMAGE_FOLDER = "images" # Replace with your image folder path

VISION_TOWER = "openai/clip-vit-large-patch14-336"
OUTPUT_DIR = "Output" # Replace with your desired output directory

In [26]:
# Command to run script
finetune_script = f'''
{DEEPSPEED_SCRIPT} \
    --lora_enable True \
    --lora_r 128 \
    --lora_alpha 256 \
    --mm_projector_lr 2e-5 \
    --bits 4 \
    --deepspeed {DEEPSPEED_JSON} \
    --model_name_or_path {MODEL_NAME} \
    --version v1 \
    --data_path {DATA_PATH} \
    --image_folder {IMAGE_FOLDER} \
    --vision_tower {VISION_TOWER} \
    --mm_projector_type mlp2x_gelu \
    --mm_vision_select_layer -2 \
    --mm_use_im_start_end False \
    --mm_use_im_patch_token False \
    --image_aspect_ratio pad \
    --group_by_modality_length True \
    --fp16 True \
    --output_dir {OUTPUT_DIR} \
    --num_train_epochs 5 \
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 1 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 50000 \
    --save_total_limit 1 \
    --learning_rate 2e-4 \
    --weight_decay 0.\
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 False \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --lazy_preprocess True \
    --report_to wandb
'''

In [41]:
import torch
torch.cuda.empty_cache()

In [ ]:
# !pip install llava

In [ ]:
!pip install flash-attn --no-build-isolation

In [ ]:
!pip install peft

In [27]:
# Execute the fibe-tuning script when bits = 4
!{finetune_script}

[2024-04-24 20:26:28,332] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible
[2024-04-24 20:26:31,166] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2024-04-24 20:26:31,167] [INFO] [runner.py:568:main] cmd = /usr/bin/python3 -

In [20]:
# Execute the fibe-tuning script
!{finetune_script}

[2024-04-24 20:19:44,162] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible
[2024-04-24 20:19:48,021] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2024-04-24 20:19:48,021] [INFO] [runner.py:568:main] cmd = /usr/bin/python3 -

Ignore Below code

In [ ]:
#!/bin/bash

# Set the prompt and model versions directly in the command
!deepspeed /LLaVA/llava/train/train_mem.py \
    --deepspeed /LLaVA/scripts/zero2.json \
    --lora_enable True \
    --lora_r 128 \
    --lora_alpha 256 \
    --mm_projector_lr 2e-5 \
    --bits 4 \
    --model_name_or_path /LLaVA/llava/llava-v1.5-7b \
    --version llava_llama_2 \
    --data_path /dataset/train/dataset.json \
    --validation_data_path /dataset/validation/dataset.json \
    --image_folder /dataset/images/ \
    --vision_tower openai/clip-vit-large-patch14-336 \
    --mm_projector_type mlp2x_gelu \
    --mm_vision_select_layer -2 \
    --mm_use_im_start_end False \
    --mm_use_im_patch_token False \
    --image_aspect_ratio pad \
    --group_by_modality_length True \
    --bf16 True \
    --output_dir /LLaVA/llava/checkpoints/llama-2-7b-chat-task-qlora \
    --num_train_epochs 500 \
    --per_device_train_batch_size 32 \
    --per_device_eval_batch_size 32 \
    --gradient_accumulation_steps 1 \
    --evaluation_strategy “epoch” \
    --save_strategy "steps" \
    --save_steps 50000 \
    --save_total_limit 1 \
    --learning_rate 2e-4 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --lazy_preprocess True \
    --report_to wandb
